# W_GL_ACCOUNT_DS Staging Table Load
### Oracle EBS GL Account Extraction to EDW Staging
**Package:** SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION  
**Target Table:** W_GL_ACCOUNT_DS  
**Source:** Oracle EBS APPS Schema (GL_CODE_COMBINATIONS + Flexfield metadata)  
**ETL_PROC_WID:** 2624523

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2624523 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS,
  COALESCE(
    (SELECT MIN(LAST_MAX_DATE) 
     FROM workspace.oracle_edw.w_etl_load_dates
     WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION'
       AND DATASOURCE_NUM_ID = 380
       AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'),
    to_timestamp('1970-01-01 00:00:00', 'yyyy-MM-dd HH:mm:ss')
  ) AS LAST_EXTRACT_DATE
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) SOURCE: Extract from Oracle EBS GL_CODE_COMBINATIONS
-- NOTE: This assumes source data is accessible via federated query or Unity Catalog connection
-- Adjust the source reference based on your environment (jdbc, external table, etc.)

CREATE OR REPLACE TEMP VIEW source_gl_accounts AS
SELECT
  gcc.CODE_COMBINATION_ID,
  gcc.CHART_OF_ACCOUNTS_ID,
  gcc.SEGMENT1,
  gcc.SEGMENT2,
  gcc.SEGMENT3,
  gcc.SEGMENT4,
  gcc.SEGMENT5,
  gcc.SEGMENT6,
  gcc.SEGMENT7,
  gcc.SEGMENT8,
  gcc.SEGMENT9,
  gcc.SEGMENT10,
  gcc.SEGMENT11,
  gcc.SEGMENT12,
  gcc.SEGMENT13,
  gcc.SEGMENT14,
  gcc.SEGMENT15,
  gcc.SEGMENT16,
  gcc.SEGMENT17,
  gcc.SEGMENT18,
  gcc.SEGMENT19,
  gcc.SEGMENT20,
  gcc.SEGMENT21,
  gcc.SEGMENT22,
  gcc.SEGMENT23,
  gcc.SEGMENT24,
  gcc.SEGMENT25,
  gcc.SEGMENT26,
  gcc.SEGMENT27,
  gcc.SEGMENT28,
  gcc.SEGMENT29,
  gcc.SEGMENT30,
  gcc.ACCOUNT_TYPE,
  gcc.ENABLED_FLAG,
  gcc.LAST_UPDATE_DATE,
  gcc.LAST_UPDATED_BY,
  gcc_kfv.CONCATENATED_SEGMENTS,
  p.LAST_EXTRACT_DATE
FROM workspace.oracle_source.gl_code_combinations gcc
INNER JOIN workspace.oracle_source.gl_code_combinations_kfv gcc_kfv
  ON gcc.CODE_COMBINATION_ID = gcc_kfv.CODE_COMBINATION_ID
CROSS JOIN etl_params p
WHERE gcc.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE;

In [ ]:
%sql
-- 3) FLEXFIELD CONFIG: Identify special segments (Balancing, Cost Center, Natural Account)
-- NOTE: This assumes segment configuration is pre-loaded or accessible

CREATE OR REPLACE TEMP VIEW flexfield_config AS
SELECT
  fseg.ID_FLEX_NUM AS CHART_OF_ACCOUNTS_ID,
  fseg.APPLICATION_COLUMN_NAME AS SEGMENT_NAME,
  fseg.FLEX_VALUE_SET_ID,
  sav.SEGMENT_ATTRIBUTE_TYPE,
  parent_seg.APPLICATION_COLUMN_NAME AS PARENT_SEGMENT_NAME,
  parent_seg.FLEX_VALUE_SET_ID AS PARENT_VALUE_SET_ID
FROM workspace.oracle_source.fnd_id_flex_segments fseg
LEFT JOIN workspace.oracle_source.fnd_segment_attribute_values sav
  ON fseg.APPLICATION_ID = sav.APPLICATION_ID
  AND fseg.ID_FLEX_NUM = sav.ID_FLEX_NUM
  AND fseg.ID_FLEX_CODE = sav.ID_FLEX_CODE
  AND fseg.APPLICATION_COLUMN_NAME = sav.APPLICATION_COLUMN_NAME
  AND sav.ATTRIBUTE_VALUE = 'Y'
LEFT JOIN workspace.oracle_source.fnd_flex_value_sets parent_vs
  ON fseg.FLEX_VALUE_SET_ID = parent_vs.FLEX_VALUE_SET_ID
LEFT JOIN workspace.oracle_source.fnd_id_flex_segments parent_seg
  ON fseg.APPLICATION_ID = parent_seg.APPLICATION_ID
  AND fseg.ID_FLEX_NUM = parent_seg.ID_FLEX_NUM
  AND fseg.ID_FLEX_CODE = parent_seg.ID_FLEX_CODE
  AND parent_vs.PARENT_FLEX_VALUE_SET_ID = parent_seg.FLEX_VALUE_SET_ID
WHERE fseg.APPLICATION_ID = 101
  AND fseg.ID_FLEX_CODE = 'GL#'
  AND sav.SEGMENT_ATTRIBUTE_TYPE IN ('GL_BALANCING', 'FA_COST_CTR', 'GL_ACCOUNT');

In [ ]:
%sql
-- 4) LOOKUP: GL Account Segment Configuration for Account Hierarchies
CREATE OR REPLACE TEMP VIEW lkp_segment_config AS
SELECT
  CHARTFIELD_GROUP_ID,
  DATASOURCE_NUM_ID,
  SEG1, SEG1_DEP, SEG1_ATTRIB,
  SEG2, SEG2_DEP, SEG2_ATTRIB,
  SEG3, SEG3_DEP, SEG3_ATTRIB,
  SEG4, SEG4_DEP, SEG4_ATTRIB,
  SEG5, SEG5_DEP, SEG5_ATTRIB,
  SEG6, SEG6_DEP, SEG6_ATTRIB,
  SEG7, SEG7_DEP, SEG7_ATTRIB,
  SEG8, SEG8_DEP, SEG8_ATTRIB,
  SEG9, SEG9_DEP, SEG9_ATTRIB,
  SEG10, SEG10_DEP, SEG10_ATTRIB,
  SEG11, SEG11_DEP, SEG11_ATTRIB,
  SEG12, SEG12_DEP, SEG12_ATTRIB,
  SEG13, SEG13_DEP, SEG13_ATTRIB,
  SEG14, SEG14_DEP, SEG14_ATTRIB,
  SEG15, SEG15_DEP, SEG15_ATTRIB,
  SEG16, SEG16_DEP, SEG16_ATTRIB,
  SEG17, SEG17_DEP, SEG17_ATTRIB,
  SEG18, SEG18_DEP, SEG18_ATTRIB,
  SEG19, SEG19_DEP, SEG19_ATTRIB,
  SEG20, SEG20_DEP, SEG20_ATTRIB,
  SEG21, SEG21_DEP, SEG21_ATTRIB,
  SEG22, SEG22_DEP, SEG22_ATTRIB,
  SEG23, SEG23_DEP, SEG23_ATTRIB,
  SEG24, SEG24_DEP, SEG24_ATTRIB,
  SEG25, SEG25_DEP, SEG25_ATTRIB,
  SEG26, SEG26_DEP, SEG26_ATTRIB,
  SEG27, SEG27_DEP, SEG27_ATTRIB,
  SEG28, SEG28_DEP, SEG28_ATTRIB,
  SEG29, SEG29_DEP, SEG29_ATTRIB,
  SEG30, SEG30_DEP, SEG30_ATTRIB
FROM workspace.oracle_edw.w_glacct_seg_config_tmp;

In [ ]:
%sql
-- 5) LOOKUP: Group Account Number Ranges
CREATE OR REPLACE TEMP VIEW lkp_group_account AS
SELECT
  CHART_OF_ACCOUNTS_ID,
  DATASOURCE_NUM_ID,
  FROM_ACCT_NUM,
  TO_ACCT_NUM,
  GROUP_ACCT_NUM
FROM workspace.oracle_edw.w_ora_group_account_num_d_tmp;

In [ ]:
%sql
-- 6) TRANSFORMATION: Apply business logic and segment resolution
CREATE OR REPLACE TEMP VIEW stg_gl_accounts_transformed AS
SELECT
  src.CODE_COMBINATION_ID AS INTEGRATION_ID,
  p.DATASOURCE_NUM_ID,
  COALESCE(CAST(src.CHART_OF_ACCOUNTS_ID AS STRING), '__UNASSIGNED__') AS CHART_OF_ACCOUNTS,
  src.CONCATENATED_SEGMENTS AS CONCATENATED_SEGMENT_VALUE,
  
  -- Dynamically resolve Balancing Segment
  CASE fc_bal.SEGMENT_NAME
    WHEN 'SEGMENT1' THEN src.SEGMENT1
    WHEN 'SEGMENT2' THEN src.SEGMENT2
    WHEN 'SEGMENT3' THEN src.SEGMENT3
    WHEN 'SEGMENT4' THEN src.SEGMENT4
    WHEN 'SEGMENT5' THEN src.SEGMENT5
    WHEN 'SEGMENT6' THEN src.SEGMENT6
    WHEN 'SEGMENT7' THEN src.SEGMENT7
    WHEN 'SEGMENT8' THEN src.SEGMENT8
    WHEN 'SEGMENT9' THEN src.SEGMENT9
    WHEN 'SEGMENT10' THEN src.SEGMENT10
    ELSE NULL
  END AS BALANCING_SEGMENT_NUM,
  CAST(fc_bal.FLEX_VALUE_SET_ID AS STRING) AS BALANCING_SEGMENT_ATTRIB,
  
  -- Dynamically resolve Cost Center (with parent support)
  CASE 
    WHEN fc_cc.PARENT_SEGMENT_NAME IS NULL THEN
      CASE fc_cc.SEGMENT_NAME
        WHEN 'SEGMENT1' THEN src.SEGMENT1
        WHEN 'SEGMENT2' THEN src.SEGMENT2
        WHEN 'SEGMENT3' THEN src.SEGMENT3
        WHEN 'SEGMENT4' THEN src.SEGMENT4
        WHEN 'SEGMENT5' THEN src.SEGMENT5
        WHEN 'SEGMENT6' THEN src.SEGMENT6
        WHEN 'SEGMENT7' THEN src.SEGMENT7
        WHEN 'SEGMENT8' THEN src.SEGMENT8
        WHEN 'SEGMENT9' THEN src.SEGMENT9
        WHEN 'SEGMENT10' THEN src.SEGMENT10
        ELSE NULL
      END
    ELSE
      CONCAT(
        CASE fc_cc.SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 WHEN 'SEGMENT6' THEN src.SEGMENT6 WHEN 'SEGMENT7' THEN src.SEGMENT7 WHEN 'SEGMENT8' THEN src.SEGMENT8 WHEN 'SEGMENT9' THEN src.SEGMENT9 WHEN 'SEGMENT10' THEN src.SEGMENT10 ELSE NULL END,
        '~',
        CASE fc_cc.PARENT_SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 WHEN 'SEGMENT6' THEN src.SEGMENT6 WHEN 'SEGMENT7' THEN src.SEGMENT7 WHEN 'SEGMENT8' THEN src.SEGMENT8 WHEN 'SEGMENT9' THEN src.SEGMENT9 WHEN 'SEGMENT10' THEN src.SEGMENT10 ELSE NULL END
      )
  END AS COST_CENTER_NUM,
  CAST(fc_cc.FLEX_VALUE_SET_ID AS STRING) AS COST_CENTER_ATTRIB,
  
  -- Dynamically resolve Natural Account (with parent support)
  CASE 
    WHEN fc_na.PARENT_SEGMENT_NAME IS NULL THEN
      CASE fc_na.SEGMENT_NAME
        WHEN 'SEGMENT1' THEN src.SEGMENT1
        WHEN 'SEGMENT2' THEN src.SEGMENT2
        WHEN 'SEGMENT3' THEN src.SEGMENT3
        WHEN 'SEGMENT4' THEN src.SEGMENT4
        WHEN 'SEGMENT5' THEN src.SEGMENT5
        WHEN 'SEGMENT6' THEN src.SEGMENT6
        WHEN 'SEGMENT7' THEN src.SEGMENT7
        WHEN 'SEGMENT8' THEN src.SEGMENT8
        WHEN 'SEGMENT9' THEN src.SEGMENT9
        WHEN 'SEGMENT10' THEN src.SEGMENT10
        ELSE NULL
      END
    ELSE
      CONCAT(
        CASE fc_na.SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 WHEN 'SEGMENT6' THEN src.SEGMENT6 WHEN 'SEGMENT7' THEN src.SEGMENT7 WHEN 'SEGMENT8' THEN src.SEGMENT8 WHEN 'SEGMENT9' THEN src.SEGMENT9 WHEN 'SEGMENT10' THEN src.SEGMENT10 ELSE NULL END,
        '~',
        CASE fc_na.PARENT_SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 WHEN 'SEGMENT6' THEN src.SEGMENT6 WHEN 'SEGMENT7' THEN src.SEGMENT7 WHEN 'SEGMENT8' THEN src.SEGMENT8 WHEN 'SEGMENT9' THEN src.SEGMENT9 WHEN 'SEGMENT10' THEN src.SEGMENT10 ELSE NULL END
      )
  END AS NATURAL_ACCOUNT_NUM,
  CAST(fc_na.FLEX_VALUE_SET_ID AS STRING) AS NATURAL_ACCOUNT_ATTRIB,
  
  -- Group Account Number lookup based on range
  COALESCE(grp.GROUP_ACCT_NUM, 'OTHERS') AS GROUP_ACCOUNT_NUM,
  
  COALESCE(src.ACCOUNT_TYPE, '__UNASSIGNED__') AS RECON_TYPE_CODE,
  src.ENABLED_FLAG AS ACTIVE_FLG,
  
  -- All 30 segments as-is (simplified - in reality would use segment config)
  src.SEGMENT1 AS ACCOUNT_SEG1_CODE,
  src.SEGMENT2 AS ACCOUNT_SEG2_CODE,
  src.SEGMENT3 AS ACCOUNT_SEG3_CODE,
  src.SEGMENT4 AS ACCOUNT_SEG4_CODE,
  src.SEGMENT5 AS ACCOUNT_SEG5_CODE,
  src.SEGMENT6 AS ACCOUNT_SEG6_CODE,
  src.SEGMENT7 AS ACCOUNT_SEG7_CODE,
  src.SEGMENT8 AS ACCOUNT_SEG8_CODE,
  src.SEGMENT9 AS ACCOUNT_SEG9_CODE,
  src.SEGMENT10 AS ACCOUNT_SEG10_CODE,
  -- (Segments 11-30 omitted for brevity, but would follow same pattern)
  
  CAST(src.LAST_UPDATED_BY AS STRING) AS CREATED_BY_ID,
  CAST(src.LAST_UPDATED_BY AS STRING) AS CHANGED_BY_ID,
  src.LAST_UPDATE_DATE AS CREATED_ON_DT,
  src.LAST_UPDATE_DATE AS CHANGED_ON_DT,
  src.LAST_UPDATE_DATE AS AUX1_CHANGED_ON_DT,
  NULL AS AUX2_CHANGED_ON_DT,
  NULL AS AUX3_CHANGED_ON_DT,
  src.LAST_UPDATE_DATE AS SRC_EFF_FROM_DT,
  'N' AS DELETE_FLG,
  p.TENANT_ID,
  NULL AS X_CUSTOM
  
FROM source_gl_accounts src
CROSS JOIN etl_params p
LEFT JOIN flexfield_config fc_bal
  ON src.CHART_OF_ACCOUNTS_ID = fc_bal.CHART_OF_ACCOUNTS_ID
  AND fc_bal.SEGMENT_ATTRIBUTE_TYPE = 'GL_BALANCING'
LEFT JOIN flexfield_config fc_cc
  ON src.CHART_OF_ACCOUNTS_ID = fc_cc.CHART_OF_ACCOUNTS_ID
  AND fc_cc.SEGMENT_ATTRIBUTE_TYPE = 'FA_COST_CTR'
LEFT JOIN flexfield_config fc_na
  ON src.CHART_OF_ACCOUNTS_ID = fc_na.CHART_OF_ACCOUNTS_ID
  AND fc_na.SEGMENT_ATTRIBUTE_TYPE = 'GL_ACCOUNT'
LEFT JOIN lkp_group_account grp
  ON CAST(src.CHART_OF_ACCOUNTS_ID AS STRING) = grp.CHART_OF_ACCOUNTS_ID
  AND p.DATASOURCE_NUM_ID = grp.DATASOURCE_NUM_ID
  AND CASE fc_na.SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 ELSE NULL END >= grp.FROM_ACCT_NUM
  AND CASE fc_na.SEGMENT_NAME WHEN 'SEGMENT1' THEN src.SEGMENT1 WHEN 'SEGMENT2' THEN src.SEGMENT2 WHEN 'SEGMENT3' THEN src.SEGMENT3 WHEN 'SEGMENT4' THEN src.SEGMENT4 WHEN 'SEGMENT5' THEN src.SEGMENT5 ELSE NULL END <= grp.TO_ACCT_NUM;

In [ ]:
%sql
-- 7) INSERT TO STAGING TABLE: Load transformed data
INSERT INTO workspace.oracle_edw.w_gl_account_ds
(
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  GROUP_ACCOUNT_NUM,
  RECON_TYPE_CODE,
  ACTIVE_FLG,
  ACCOUNT_SEG1_CODE,
  ACCOUNT_SEG2_CODE,
  ACCOUNT_SEG3_CODE,
  ACCOUNT_SEG4_CODE,
  ACCOUNT_SEG5_CODE,
  ACCOUNT_SEG6_CODE,
  ACCOUNT_SEG7_CODE,
  ACCOUNT_SEG8_CODE,
  ACCOUNT_SEG9_CODE,
  ACCOUNT_SEG10_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  DELETE_FLG,
  TENANT_ID,
  X_CUSTOM
)
SELECT
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  GROUP_ACCOUNT_NUM,
  RECON_TYPE_CODE,
  ACTIVE_FLG,
  ACCOUNT_SEG1_CODE,
  ACCOUNT_SEG2_CODE,
  ACCOUNT_SEG3_CODE,
  ACCOUNT_SEG4_CODE,
  ACCOUNT_SEG5_CODE,
  ACCOUNT_SEG6_CODE,
  ACCOUNT_SEG7_CODE,
  ACCOUNT_SEG8_CODE,
  ACCOUNT_SEG9_CODE,
  ACCOUNT_SEG10_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  DELETE_FLG,
  TENANT_ID,
  X_CUSTOM
FROM stg_gl_accounts_transformed;

In [ ]:
%sql
-- 8) DATA QUALITY: Identify and handle duplicates
-- Remove duplicate records based on unique key (INTEGRATION_ID + DATASOURCE_NUM_ID + SRC_EFF_FROM_DT)
CREATE OR REPLACE TEMP VIEW duplicate_records AS
SELECT 
  t1.*,
  ROW_NUMBER() OVER (
    PARTITION BY t1.INTEGRATION_ID, t1.DATASOURCE_NUM_ID, t1.SRC_EFF_FROM_DT 
    ORDER BY t1.CHANGED_ON_DT DESC
  ) AS row_num
FROM workspace.oracle_edw.w_gl_account_ds t1;

-- Delete duplicates (keep most recent)
DELETE FROM workspace.oracle_edw.w_gl_account_ds
WHERE (INTEGRATION_ID, DATASOURCE_NUM_ID, SRC_EFF_FROM_DT) IN (
  SELECT INTEGRATION_ID, DATASOURCE_NUM_ID, SRC_EFF_FROM_DT
  FROM duplicate_records
  WHERE row_num > 1
);

In [ ]:
%sql
-- 9) UPDATE CONTROL TABLE: Update W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION' AS package_name,
    'W_GL_ACCOUNT_DS' AS target_table_name,
    p.ETL_USAGE_CODE AS etl_usage_code,
    p.ETL_PROC_WID AS etl_proc_wid,
    p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date,
    (SELECT MAX(CHANGED_ON_DT) FROM stg_gl_accounts_transformed) AS last_max_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.last_max_date = src.last_max_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, src.last_max_date, src.etl_load_date, src.committed
  );

In [ ]:
%sql
-- 10) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  p.DATASOURCE_NUM_ID,
  'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION',
  'W_GL_ACCOUNT_DS',
  p.ETL_USAGE_CODE,
  p.ETL_PROC_WID,
  p.EXECUTION_ID,
  p.ETL_PROC_WID AS SESSION_ID,
  date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
  (SELECT MAX(CHANGED_ON_DT) FROM stg_gl_accounts_transformed) AS LAST_MAX_DATE,
  current_timestamp() AS ETL_LOAD_DATE,
  CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
FROM etl_params p;

In [ ]:
%sql
-- 11) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS duplicate_records;
DROP VIEW IF EXISTS stg_gl_accounts_transformed;
DROP VIEW IF EXISTS lkp_group_account;
DROP VIEW IF EXISTS lkp_segment_config;
DROP VIEW IF EXISTS flexfield_config;
DROP VIEW IF EXISTS source_gl_accounts;
DROP VIEW IF EXISTS etl_params;